In [1]:
import trimesh

scene = trimesh.Scene()

In [2]:
import pyvirtualdisplay
import trimesh
import my_code.diffusion_training_sign_corr.data_loading as data_loading
import yaml
import json
from tqdm import tqdm
import torch
import numpy as np
import trimesh.scene
import trimesh.scene.lighting
import plotly.express as px
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import os
import PIL.Image



dataset_name = 'SMAL_nocat_pair'

single_dataset, pair_dataset = data_loading.get_val_dataset(
    dataset_name, 'test', 128, preload=False, return_evecs=True, centering='bbox'
)


if dataset_name == 'SHREC19_r_pair':
    file_name = '/lustre/mlnvme/data/s94zalek_hpc-shape_matching/ddpm_checkpoints/single_64_1-2ev_64-128-128_remeshed_fixed/eval/epoch_99/SHREC19_r_pair-test/no_smoothing/2024-11-04_22-27-59/pairwise_results.json'
elif dataset_name == 'DT4D_intra_pair':
    file_name = '/lustre/mlnvme/data/s94zalek_hpc-shape_matching/ddpm_checkpoints/single_template_remeshed/eval/checkpoint_99.pt/DT4D_intra_pair-test/no_smoothing/2024-11-10_21-20-05/pairwise_results.json'
elif dataset_name == 'DT4D_inter_pair':
    file_name = '/lustre/mlnvme/data/s94zalek_hpc-shape_matching/ddpm_checkpoints/single_template_remeshed/eval/checkpoint_99.pt/DT4D_inter_pair-test/no_smoothing/2024-11-10_21-20-05/pairwise_results.json'
elif dataset_name == 'FAUST_r_pair':
    file_name = '/lustre/mlnvme/data/s94zalek_hpc-shape_matching/ddpm_checkpoints/single_64_1-2ev_64-128-128_remeshed_fixed/eval/epoch_99/FAUST_r_pair-test/no_smoothing/2024-11-04_22-27-59/pairwise_results.json'
elif dataset_name == 'SCAPE_r_pair':
    file_name = '/lustre/mlnvme/data/s94zalek_hpc-shape_matching/ddpm_checkpoints/single_64_1-2ev_64-128-128_remeshed_fixed/eval/epoch_99/SCAPE_r_pair-test/no_smoothing/2024-11-04_22-27-59/pairwise_results.json'
elif dataset_name == 'SMAL_nocat_pair':
    file_name = '/lustre/mlnvme/data/s94zalek_hpc-shape_matching/ddpm_checkpoints/single_64_SMAL_nocat_64_SMAL_isoRemesh_0.2_0.8_nocat_1-2ev_64k/eval/epoch_99/SMAL_nocat_pair-test/no_smoothing/2025-01-24_16-01-31/pairwise_results.json'
        
with open(file_name, 'r') as f:
    p2p_saved = json.load(f)


geo_err_list = torch.tensor([p2p_saved[i]['geo_err_median_pairzo'] for i in range(len(p2p_saved))])
idxs_geo_err = torch.argsort(geo_err_list, descending=True)

        

In [3]:
indx = 38

data_i = pair_dataset[indx]
p2p_i = p2p_saved[indx]
p2p_pairzo = torch.tensor(p2p_i['p2p_median_pairzo'])

print(p2p_i['geo_err_median_pairzo'])

Cxy = torch.linalg.lstsq(
    data_i['second']['evecs'],
    data_i['first']['evecs'][p2p_pairzo],
).solution

Pyx = data_i['second']['evecs'] @ Cxy @ data_i['first']['evecs_trans']




In [ ]:
# read /home/s94zalek_hpc/shape_matching/figures/texture.png with PIL
from PIL import Image
import utils.texture_util as texture_util


#########################################
# X -> Y
#########################################

scene.geometry.clear()

# verts_x = data_i['first']['verts'].clone()
# verts_y = data_i['second']['verts'].clone()

# verts_x_cloned = verts_x.clone()

# verts_x[:, 0] = verts_x_cloned[:, 2]
# verts_x[:, 1] = -verts_x_cloned[:, 1]
# verts_x[:, 2] = verts_x_cloned[:, 0]

# verts_y_cloned = verts_y.clone()

# verts_y[:, 0] = verts_y_cloned[:, 2]
# verts_y[:, 1] = -verts_y_cloned[:, 1]
# verts_y[:, 2] = verts_y_cloned[:, 0]


# texture_img = Image.open('/home/s94zalek_hpc/shape_matching/figures/texture.png')
# texture_img = Image.open('/home/s94zalek_hpc/shape_matching/figures/Custom_texture.png')
texture_img = Image.open('/home/s94zalek_hpc/shape_matching/figures/CustomUVChecker_byValle_2K (7).png')

# create material
material=trimesh.visual.material.SimpleMaterial(
        image=texture_img,
        diffuse=[255, 255, 255, 255],
    )

# add the first mesh
# mesh1 = trimesh.Trimesh(
#     vertices=data_i['first']['verts'].numpy(), 
#     faces=data_i['first']['faces'].numpy(),
#     process=False, validate=False,
#     )

mesh1 = trimesh.Trimesh(
    vertices=data_i['first']['verts'].numpy(), 
    faces=data_i['first']['faces'].numpy(),
    process=False, validate=False,
    )

# print(mesh1.vertices)

rot_mat = trimesh.transformations.rotation_matrix(np.pi, [1, 0, 0], [0, 0, 0])
mesh1.apply_transform(rot_mat)
mesh1.apply_transform(trimesh.transformations.rotation_matrix(-np.pi/3, [0, 1, 0], [0, 0, 0]))

# print(mesh1.vertices)

uv1 = texture_util.generate_tex_coords(mesh1.vertices, col1=0, col2=1) * 1.5
uv2 = Pyx @ uv1


texture_visuals = trimesh.visual.texture.TextureVisuals(
    uv=uv1[:len(mesh1.vertices)],
    material=material
)

mesh1.visual = texture_visuals




# correspondence we got from the functional map
mesh2_after = trimesh.Trimesh(
    vertices=data_i['second']['verts'].numpy() + np.array([1, 0, 0]), 
    faces=data_i['second']['faces'].numpy(),
    process=False, validate=False,
    )
mesh2_after.apply_transform(rot_mat)
mesh2_after.apply_transform(trimesh.transformations.rotation_matrix(-6*np.pi/6, [0, 1, 0], [1, 0, 0]))

texture_visuals_after = trimesh.visual.texture.TextureVisuals(
    uv=uv2[:len(mesh2_after.vertices)],
    material=material
)
mesh2_after.visual = texture_visuals_after


# scene.add_geometry(mesh1)
scene.add_geometry(mesh2_after)

scene.set_camera()
scene.show()

In [5]:
base_path = '/lustre/mlnvme/data/s94zalek_hpc-shape_matching/figures/p2p_texture'
target_path = f'{base_path}/{dataset_name}/combined'

os.makedirs(target_path, exist_ok=True)

with pyvirtualdisplay.Display(visible=False, size=(1920, 1080)) as disp:
    png = scene.save_image(resolution=(1920, 1080), visible=True)

# png = scene.save_image(resolution=(int(1920*1080), 1080), visible=True)

with open(f'{target_path}/002.png', "wb") as f:
    f.write(png)

In [6]:
# for i, combination in enumerate(pair_dataset.combinations):
#     print(f'{i}: {single_dataset.off_files[combination[0]].split("/")[-1].split(".")[0]} - {single_dataset.off_files[combination[1]].split("/")[-1].split(".")[0]}')